In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [2]:
# import datasets
names = ['age','workclass','fnlwgt','education','education-num','marital-status',
         'occupation','relationship','race','sex','capital-gain','capital-loss',
         'hours-per-week','native-country','income']
train = pd.read_csv('../Data/adult.data',names=names,index_col=False)
test = pd.read_csv('../Data/adult.test',names=names,index_col=False,skiprows=1) # first row in the test dataset is irrelevant

In [3]:
train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
test.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [5]:
# remove space and dot in data
def remove_space_dot(train,test):
    for col in ['workclass','education','marital-status','occupation','relationship','race','sex','native-country','income']:
        train[col] = train[col].str.strip()
        test[col] = test[col].str.strip()
        if col == 'income':
            test[col] = test[col].str.replace('.','')
    return train,test

train,test = remove_space_dot(train,test)

In [6]:
# transform income into binary values
def transform_income(train,test):
    train.loc[:,'income'] = np.where(train.loc[:,'income']=='>50K',1,0)
    test.loc[:,'income'] = np.where(test.loc[:,'income']=='>50K',1,0)
    return train,test

train,test = transform_income(train,test)

In [7]:
# sanity_check
test[test['income']==1]

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,1
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,1
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,1
10,65,Private,184454,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,6418,0,40,United-States,1
14,48,Private,279724,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,3103,0,48,United-States,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16254,38,Private,149347,Masters,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,1
16255,43,Local-gov,23157,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1902,50,United-States,1
16261,40,Private,202168,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,15024,0,55,United-States,1
16265,50,Local-gov,139347,Masters,14,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,0,40,?,1


In [8]:
# since missing values are written as ?, replace them with NaN
def fill_nan(train,test):
    train.replace('?',np.nan,inplace=True)
    test.replace('?',np.nan,inplace=True)
    return train,test

train,test = fill_nan(train,test)

print(train.isnull().sum(),'\n')
print(test.isnull().sum())

age                  0
workclass         1836
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
income               0
dtype: int64 

age                 0
workclass         963
fnlwgt              0
education           0
education-num       0
marital-status      0
occupation        966
relationship        0
race                0
sex                 0
capital-gain        0
capital-loss        0
hours-per-week      0
native-country    274
income              0
dtype: int64


In [9]:
# since the number of null values are quite small to the total number of instances, let's just drop them from the datasets
def drop_nan(train,test):
    train.dropna(axis=0,how='any',inplace=True)
    test.dropna(axis=0,how='any',inplace=True)
    return train,test

train,test = drop_nan(train,test)

print(train.isnull().sum(),'\n')
print(test.isnull().sum())

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64 

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64


In [10]:
train.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,income
count,30162.000000,3.016200e+04,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000
mean,38.437902,1.897938e+05,10.121312,1092.007858,88.372489,40.931238,0.248922
std,13.134665,1.056530e+05,2.549995,7406.346497,404.298370,11.979984,0.432396
min,17.000000,1.376900e+04,1.000000,0.000000,0.000000,1.000000,0.000000
25%,28.000000,1.176272e+05,9.000000,0.000000,0.000000,40.000000,0.000000
50%,37.000000,1.784250e+05,10.000000,0.000000,0.000000,40.000000,0.000000
75%,47.000000,2.376285e+05,13.000000,0.000000,0.000000,45.000000,0.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000,1.000000


In [11]:
test.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,income
count,15060.000000,1.506000e+04,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000
mean,38.768327,1.896164e+05,10.112749,1120.301594,89.041899,40.951594,0.245684
std,13.380676,1.056150e+05,2.558727,7703.181842,406.283245,12.062831,0.430506
min,17.000000,1.349200e+04,1.000000,0.000000,0.000000,1.000000,0.000000
25%,28.000000,1.166550e+05,9.000000,0.000000,0.000000,40.000000,0.000000
50%,37.000000,1.779550e+05,10.000000,0.000000,0.000000,40.000000,0.000000
75%,48.000000,2.385888e+05,13.000000,0.000000,0.000000,45.000000,0.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,3770.000000,99.000000,1.000000


In [12]:
"""
## Doing preprocessing on each column
* Numerical values -> standardize
* Categorical values -> if ordinal, transform them into ordinal integers (e.g. 3,2,1,0)... if cardinal, combine them into fewer number of categories and one-hot-encode them
* Binary classes -> turn them into 0, 1
"""
# age  - standardize
def process_age(train,test):
    age_scaler = StandardScaler()
    age_scaler.fit(np.array(train.loc[:,'age']).reshape(-1,1)) # fit the scaler into train data
    train.age = age_scaler.transform(np.array(train.loc[:,'age']).reshape(-1,1)) 
    test.age = age_scaler.transform(np.array(test.loc[:,'age']).reshape(-1,1))
    return train,test

train,test = process_age(train,test)

In [13]:
# sanity check on age column
print(train.age.describe(),'\n')
print(test.age.describe(),'\n')

count    3.016200e+04
mean     1.535952e-16
std      1.000017e+00
min     -1.632189e+00
25%     -7.946967e-01
50%     -1.094756e-01
75%      6.518811e-01
max      3.925715e+00
Name: age, dtype: float64 

count    15060.000000
mean         0.025157
std          1.018747
min         -1.632189
25%         -0.794697
50%         -0.109476
75%          0.728017
max          3.925715
Name: age, dtype: float64 



In [14]:
# workclass - Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
# see counts of each value
train.workclass.value_counts()

Private             22286
Self-emp-not-inc     2499
Local-gov            2067
State-gov            1279
Self-emp-inc         1074
Federal-gov           943
Without-pay            14
Name: workclass, dtype: int64

In [15]:
"""
One-hot-encode workclass column into...
1. Private
2. Self-emp (Self-emp-not-inc + Self-emp-inc)
3. gov (Local-gov + State-gov + Federal-gov)
4. Without-pay
5. Neverworked
"""

def process_workclass(train,test):
    # Self-emp
    train.loc[:,'workclass'] = np.where(train.loc[:,'workclass'].isin(['Self-emp-not-inc','Self-emp-inc']),'Self-emp',train.loc[:,'workclass'])
    test.loc[:,'workclass'] = np.where(test.loc[:,'workclass'].isin(['Self-emp-not-inc','Self-emp-inc']),'Self-emp',test.loc[:,'workclass'])
    # gov
    train.loc[:,'workclass'] = np.where(train.loc[:,'workclass'].isin(['Local-gov','State-gov','Federal-gov']),'Gov',train.loc[:,'workclass'])
    test.loc[:,'workclass'] = np.where(test.loc[:,'workclass'].isin(['Local-gov','State-gov','Federal-gov']),'Gov',test.loc[:,'workclass'])
    return train,test

train,test = process_workclass(train,test)

In [16]:
# sanity check
print(train.workclass.value_counts(),'\n')
print(test.workclass.value_counts(),'\n')

Private        22286
Gov             4289
Self-emp        3573
Without-pay       14
Name: workclass, dtype: int64 

Private        11021
Gov             2163
Self-emp        1869
Without-pay        7
Name: workclass, dtype: int64 



In [17]:
# one-hot-encode workclass
def one_hot_encode_workclass(train,test):
    # one-hot-encode
    train_temp = pd.get_dummies(train.workclass)
    test_temp = pd.get_dummies(test.workclass)
    # concat to original dataset
    new_train = pd.concat([train,train_temp],axis=1)
    new_test = pd.concat([test,test_temp],axis=1)
    # drop original workclass column
    new_train.drop('workclass',axis=1,inplace=True)
    new_test.drop('workclass',axis=1,inplace=True)
    return new_train,new_test

train,test = one_hot_encode_workclass(train,test)

In [18]:
# sanity check
train.head()

,age,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income,Gov,Private,Self-emp,Without-pay
0,0.042796,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0,1,0,0,0
1,0.880288,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0,0,0,1,0
2,-0.033340,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0,0,1,0,0
3,1.108695,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0,0,1,0,0
4,-0.794697,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0,0,1,0,0


In [19]:
def transform_fnlwgt(train,test):
    fnlwgt_scaler = StandardScaler()
    fnlwgt_scaler.fit(np.array(train.loc[:,'fnlwgt']).reshape(-1,1))
    train.fnlwgt = fnlwgt_scaler.transform(np.array(train.loc[:,'fnlwgt']).reshape(-1,1))
    test.fnlwgt = fnlwgt_scaler.transform(np.array(test.loc[:,'fnlwgt']).reshape(-1,1))
    return train,test

train,test = transform_fnlwgt(train,test)

In [20]:
# sanity check
print(train.fnlwgt.describe(),'\n')
print(test.fnlwgt.describe(),'\n')

count    3.016200e+04
mean     1.684365e-17
std      1.000017e+00
min     -1.666094e+00
25%     -6.830644e-01
50%     -1.076072e-01
75%      4.527602e-01
max      1.225647e+01
Name: fnlwgt, dtype: float64 

count    15060.000000
mean        -0.001680
std          0.999657
min         -1.668716
25%         -0.692267
50%         -0.112056
75%          0.461849
max         12.310375
Name: fnlwgt, dtype: float64 



In [21]:
# education - Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
# see counts of each value
train.education.value_counts()

HS-grad         9840
Some-college    6678
Bachelors       5044
Masters         1627
Assoc-voc       1307
11th            1048
Assoc-acdm      1008
10th             820
7th-8th          557
Prof-school      542
9th              455
12th             377
Doctorate        375
5th-6th          288
1st-4th          151
Preschool         45
Name: education, dtype: int64

In [22]:
"""
Transform education column into ordinal integer values from 0 to 6...
1. Doctorate
2. Masters
3. Bachelors
4. College (Some-college+Assoc-voc+Assoc-acdm+Prof-school)
5. HS-grad
6. Under-HS (12th~1st)
7. Preschool
"""

def process_education(train,test):
    # Some-college
    train.loc[:,'education'] = np.where(train.loc[:,'education'].isin(['Some-college','Assoc-voc','Assoc-acdm','Prof-school']),'College',train.loc[:,'education'])
    test.loc[:,'education'] = np.where(test.loc[:,'education'].isin(['Some-college','Assoc-voc','Assoc-acdm','Prof-school']),'College',test.loc[:,'education'])
    # Under-HS
    train.loc[:,'education'] = np.where(train.loc[:,'education'].isin(['12th','11th','10th','9th','7th-8th','5th-6th','1st-4th']),'Under-HS',train.loc[:,'education'])
    test.loc[:,'education'] = np.where(test.loc[:,'education'].isin(['12th','11th','10th','9th','7th-8th','5th-6th','1st-4th']),'Under-HS',test.loc[:,'education'])
    return train,test

train,test = process_education(train,test)

In [23]:
# sanity check on education
train.education.value_counts()

HS-grad      9840
College      9535
Bachelors    5044
Under-HS     3696
Masters      1627
Doctorate     375
Preschool      45
Name: education, dtype: int64

In [24]:
train['education']

0        Bachelors
1        Bachelors
2          HS-grad
3         Under-HS
4        Bachelors
           ...    
32556      College
32557      HS-grad
32558      HS-grad
32559      HS-grad
32560      HS-grad
Name: education, Length: 30162, dtype: object

In [25]:
# transform into ordinal integers and then standardize again
def transform_education(train,test):
    # transform
    edu_list = ['Preschool','Under-HS','HS-grad','College','Bachelors','Masters','Doctorate']
    train.loc[:,'education'] = train.loc[:,'education'].apply(lambda x: edu_list.index(x))
    test.loc[:,'education'] = test.loc[:,'education'].apply(lambda x: edu_list.index(x))
    # standardize
    edu_scaler = StandardScaler()
    edu_scaler.fit(np.array(train.loc[:,'education']).reshape(-1,1))
    train.education = edu_scaler.transform(np.array(train.loc[:,'education']).reshape(-1,1))
    test.education = edu_scaler.transform(np.array(test.loc[:,'education']).reshape(-1,1))
    return train,test

train,test = transform_education(train,test)

In [26]:
# sanity check
train.education.value_counts()

-0.657451    9840
 0.235070    9535
 1.127591    5044
-1.549972    3696
 2.020112    1627
 2.912633     375
-2.442493      45
Name: education, dtype: int64

In [27]:
# education_num 
# see counts of each value
train['education-num'].value_counts()

9     9840
10    6678
13    5044
14    1627
11    1307
7     1048
12    1008
6      820
4      557
15     542
5      455
8      377
16     375
3      288
2      151
1       45
Name: education-num, dtype: int64

In [28]:
# Since this column looks almost identical to Education, let's just drop this
train.drop('education-num',axis=1,inplace=True)
test.drop('education-num',axis=1,inplace=True)
train.head()

,age,fnlwgt,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income,Gov,Private,Self-emp,Without-pay
0,0.042796,-1.062722,1.127591,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0,1,0,0,0
1,0.880288,-1.007871,1.127591,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0,0,0,1,0
2,-0.033340,0.244693,-0.657451,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0,0,1,0,0
3,1.108695,0.425240,-1.549972,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0,0,1,0,0
4,-0.794697,1.406658,1.127591,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0,0,1,0,0


In [29]:
# marital-status - Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
# see counts of each value
train['marital-status'].value_counts()

Married-civ-spouse       14065
Never-married             9726
Divorced                  4214
Separated                  939
Widowed                    827
Married-spouse-absent      370
Married-AF-spouse           21
Name: marital-status, dtype: int64

In [30]:
"""
One-hot-encode marital-status column...
First, combine them into...
1. Married (Married-civ-spouse+Married-spoucse-absent+Married-AF-spouse)
2. Never-married
3. Not-married (Divorced+Separated+Widowed)
"""
def process_marital(train,test):
    # Married
    train.loc[:,'marital-status'] = np.where(train.loc[:,'marital-status'].isin(['Married-civ-spouse','Married-spouse-absent','Married-AF-spouse']),'Married',train.loc[:,'marital-status'])
    test.loc[:,'marital-status'] = np.where(test.loc[:,'marital-status'].isin(['Married-civ-spouse','Married-spouse-absent','Married-AF-spouse']),'Married',test.loc[:,'marital-status'])
    # Not-married
    train.loc[:,'marital-status'] = np.where(train.loc[:,'marital-status'].isin(['Divorced','Widowed','Separated']),'Not-married',train.loc[:,'marital-status'])
    test.loc[:,'marital-status'] = np.where(test.loc[:,'marital-status'].isin(['Divorced','Widowed','Separated']),'Not-married',test.loc[:,'marital-status'])
    return train,test

# one-hot-encode marital-status
def one_hot_encode_marital(train,test):
    # one-hot-encode
    train_temp = pd.get_dummies(train['marital-status'])
    test_temp = pd.get_dummies(test['marital-status'])
    # concat to original dataset
    new_train = pd.concat([train,train_temp],axis=1)
    new_test = pd.concat([test,test_temp],axis=1)
    # drop original workclass column
    new_train.drop('marital-status',axis=1,inplace=True)
    new_test.drop('marital-status',axis=1,inplace=True)
    return new_train,new_test

train,test = process_marital(train,test)
train,test = one_hot_encode_marital(train,test)

In [31]:
# sanity check
train.head()

,age,fnlwgt,education,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income,Gov,Private,Self-emp,Without-pay,Married,Never-married,Not-married
0,0.042796,-1.062722,1.127591,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0,1,0,0,0,0,1,0
1,0.880288,-1.007871,1.127591,Exec-managerial,Husband,White,Male,0,0,13,United-States,0,0,0,1,0,1,0,0
2,-0.033340,0.244693,-0.657451,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0,0,1,0,0,0,0,1
3,1.108695,0.425240,-1.549972,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0,0,1,0,0,1,0,0
4,-0.794697,1.406658,1.127591,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0,0,1,0,0,1,0,0


In [32]:
# occupation - Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
# see counts of each value
train['occupation'].value_counts()

Prof-specialty       4038
Craft-repair         4030
Exec-managerial      3992
Adm-clerical         3721
Sales                3584
Other-service        3212
Machine-op-inspct    1966
Transport-moving     1572
Handlers-cleaners    1350
Farming-fishing       989
Tech-support          912
Protective-serv       644
Priv-house-serv       143
Armed-Forces            9
Name: occupation, dtype: int64

In [33]:
"""
One-hot-encode occupation column...
Seems like it's difficult to combine them into fewer number of jobs.
"""

# one-hot-encode occupation
def one_hot_encode_occupation(train,test):
    # one-hot-encode
    train_temp = pd.get_dummies(train['occupation'])
    test_temp = pd.get_dummies(test['occupation'])
    # concat to original dataset
    new_train = pd.concat([train,train_temp],axis=1)
    new_test = pd.concat([test,test_temp],axis=1)
    # drop original workclass column
    new_train.drop('occupation',axis=1,inplace=True)
    new_test.drop('occupation',axis=1,inplace=True)
    return new_train,new_test

train,test = one_hot_encode_occupation(train,test)

In [34]:
# sanity check
train.head()

,age,fnlwgt,education,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,...,Farming-fishing,Handlers-cleaners,Machine-op-inspct,Other-service,Priv-house-serv,Prof-specialty,Protective-serv,Sales,Tech-support,Transport-moving
0,0.042796,-1.062722,1.127591,Not-in-family,White,Male,2174,0,40,United-States,...,0,0,0,0,0,0,0,0,0,0
1,0.880288,-1.007871,1.127591,Husband,White,Male,0,0,13,United-States,...,0,0,0,0,0,0,0,0,0,0
2,-0.033340,0.244693,-0.657451,Not-in-family,White,Male,0,0,40,United-States,...,0,1,0,0,0,0,0,0,0,0
3,1.108695,0.425240,-1.549972,Husband,Black,Male,0,0,40,United-States,...,0,1,0,0,0,0,0,0,0,0
4,-0.794697,1.406658,1.127591,Wife,Black,Female,0,0,40,Cuba,...,0,0,0,0,0,1,0,0,0,0


In [35]:
# relationship - Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
# see counts of each value
train['relationship'].value_counts()

Husband           12463
Not-in-family      7726
Own-child          4466
Unmarried          3212
Wife               1406
Other-relative      889
Name: relationship, dtype: int64

In [36]:
"""
One-hot-encoding relationship will lead to too many sparse attributes,
and this column seems redundant/similar to marital status.
Decided to drop this.
"""
# Since this column looks almost identical to Education, let's just drop this
train.drop('relationship',axis=1,inplace=True)
test.drop('relationship',axis=1,inplace=True)
train.head()

,age,fnlwgt,education,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income,...,Farming-fishing,Handlers-cleaners,Machine-op-inspct,Other-service,Priv-house-serv,Prof-specialty,Protective-serv,Sales,Tech-support,Transport-moving
0,0.042796,-1.062722,1.127591,White,Male,2174,0,40,United-States,0,...,0,0,0,0,0,0,0,0,0,0
1,0.880288,-1.007871,1.127591,White,Male,0,0,13,United-States,0,...,0,0,0,0,0,0,0,0,0,0
2,-0.033340,0.244693,-0.657451,White,Male,0,0,40,United-States,0,...,0,1,0,0,0,0,0,0,0,0
3,1.108695,0.425240,-1.549972,Black,Male,0,0,40,United-States,0,...,0,1,0,0,0,0,0,0,0,0
4,-0.794697,1.406658,1.127591,Black,Female,0,0,40,Cuba,0,...,0,0,0,0,0,1,0,0,0,0


In [37]:
# race - White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
# see counts of each value
train['race'].value_counts()

White                 25933
Black                  2817
Asian-Pac-Islander      895
Amer-Indian-Eskimo      286
Other                   231
Name: race, dtype: int64

In [38]:
"""
Transform race into binary values indicating whether the person is white or not,
due to significant class imbalance & bias towards white.
"""
def transform_race(train,test):
    train.loc[:,'race'] = np.where(train.loc[:,'race']=='White',1,0)
    test.loc[:,'race'] = np.where(test.loc[:,'race']=='White',1,0)
    return train,test

train,test = transform_race(train,test)

In [39]:
# sanity check
train.head()

,age,fnlwgt,education,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income,...,Farming-fishing,Handlers-cleaners,Machine-op-inspct,Other-service,Priv-house-serv,Prof-specialty,Protective-serv,Sales,Tech-support,Transport-moving
0,0.042796,-1.062722,1.127591,1,Male,2174,0,40,United-States,0,...,0,0,0,0,0,0,0,0,0,0
1,0.880288,-1.007871,1.127591,1,Male,0,0,13,United-States,0,...,0,0,0,0,0,0,0,0,0,0
2,-0.033340,0.244693,-0.657451,1,Male,0,0,40,United-States,0,...,0,1,0,0,0,0,0,0,0,0
3,1.108695,0.425240,-1.549972,0,Male,0,0,40,United-States,0,...,0,1,0,0,0,0,0,0,0,0
4,-0.794697,1.406658,1.127591,0,Female,0,0,40,Cuba,0,...,0,0,0,0,0,1,0,0,0,0


In [40]:
"""
Transform sex into binary values.
"""
def transform_sex(train,test):
    train.loc[:,'sex'] = np.where(train.loc[:,'sex']=='Female',1,0)
    test.loc[:,'sex'] = np.where(test.loc[:,'sex']=='Female',1,0)
    return train,test

train,test = transform_sex(train,test)

In [41]:
# sanity check
train.head()

,age,fnlwgt,education,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income,...,Farming-fishing,Handlers-cleaners,Machine-op-inspct,Other-service,Priv-house-serv,Prof-specialty,Protective-serv,Sales,Tech-support,Transport-moving
0,0.042796,-1.062722,1.127591,1,0,2174,0,40,United-States,0,...,0,0,0,0,0,0,0,0,0,0
1,0.880288,-1.007871,1.127591,1,0,0,0,13,United-States,0,...,0,0,0,0,0,0,0,0,0,0
2,-0.033340,0.244693,-0.657451,1,0,0,0,40,United-States,0,...,0,1,0,0,0,0,0,0,0,0
3,1.108695,0.425240,-1.549972,0,0,0,0,40,United-States,0,...,0,1,0,0,0,0,0,0,0,0
4,-0.794697,1.406658,1.127591,0,1,0,0,40,Cuba,0,...,0,0,0,0,0,1,0,0,0,0


In [42]:
# capital-gain & loss
print(train['capital-gain'].describe(),'\n')
print(train['capital-loss'].describe(),'\n')
print(test['capital-gain'].describe(),'\n')
print(test['capital-loss'].describe(),'\n')

count    30162.000000
mean      1092.007858
std       7406.346497
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max      99999.000000
Name: capital-gain, dtype: float64 

count    30162.000000
mean        88.372489
std        404.298370
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max       4356.000000
Name: capital-loss, dtype: float64 

count    15060.000000
mean      1120.301594
std       7703.181842
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max      99999.000000
Name: capital-gain, dtype: float64 

count    15060.000000
mean        89.041899
std        406.283245
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max       3770.000000
Name: capital-loss, dtype: float64 



In [43]:
"""
Standardize capital-gain and capital-loss.
"""
def transform_capital(train,test):
    # capital-gain
    capital_gain_scaler = StandardScaler()
    capital_gain_scaler.fit(np.array(train.loc[:,'capital-gain']).reshape(-1,1))
    train['capital-gain'] = capital_gain_scaler.transform(np.array(train.loc[:,'capital-gain']).reshape(-1,1))
    test['capital-gain'] = capital_gain_scaler.transform(np.array(test.loc[:,'capital-gain']).reshape(-1,1))
    # capital-loss
    capital_loss_scaler = StandardScaler()
    capital_loss_scaler.fit(np.array(train.loc[:,'capital-loss']).reshape(-1,1))
    train['capital-loss'] = capital_loss_scaler.transform(np.array(train.loc[:,'capital-loss']).reshape(-1,1))
    test['capital-loss'] = capital_loss_scaler.transform(np.array(test.loc[:,'capital-loss']).reshape(-1,1))
    return train,test

train,test = transform_capital(train,test)

In [44]:
# sanity check
print(train['capital-gain'].describe(),'\n')
print(train['capital-loss'].describe())

count    3.016200e+04
mean    -2.720897e-17
std      1.000017e+00
min     -1.474446e-01
25%     -1.474446e-01
50%     -1.474446e-01
75%     -1.474446e-01
max      1.335458e+01
Name: capital-gain, dtype: float64 

count    3.016200e+04
mean     7.703318e-17
std      1.000017e+00
min     -2.185860e-01
25%     -2.185860e-01
50%     -2.185860e-01
75%     -2.185860e-01
max      1.055581e+01
Name: capital-loss, dtype: float64


In [45]:
# hours-per-week
train['hours-per-week'].describe()

count    30162.000000
mean        40.931238
std         11.979984
min          1.000000
25%         40.000000
50%         40.000000
75%         45.000000
max         99.000000
Name: hours-per-week, dtype: float64

In [46]:
"""
Standardize hours-per-week
"""
def transform_hours(train,test):
    # hours-per-week
    hours_scaler = StandardScaler()
    hours_scaler.fit(np.array(train.loc[:,'hours-per-week']).reshape(-1,1))
    train['hours-per-week'] = hours_scaler.transform(np.array(train.loc[:,'hours-per-week']).reshape(-1,1))
    test['hours-per-week'] = hours_scaler.transform(np.array(test.loc[:,'hours-per-week']).reshape(-1,1))
    return train,test

train,test = transform_hours(train,test)

In [47]:
# sanity check
print(train['hours-per-week'].describe())

count    3.016200e+04
mean    -2.833973e-16
std      1.000017e+00
min     -3.333218e+00
25%     -7.773411e-02
50%     -7.773411e-02
75%      3.396356e-01
max      4.847229e+00
Name: hours-per-week, dtype: float64


In [48]:
# native-country - United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, 
# Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
# see value counts
train['native-country'].value_counts()

United-States                 27504
Mexico                          610
Philippines                     188
Germany                         128
Puerto-Rico                     109
Canada                          107
India                           100
El-Salvador                     100
Cuba                             92
England                          86
Jamaica                          80
South                            71
China                            68
Italy                            68
Dominican-Republic               67
Vietnam                          64
Guatemala                        63
Japan                            59
Columbia                         56
Poland                           56
Iran                             42
Haiti                            42
Taiwan                           42
Portugal                         34
Nicaragua                        33
Peru                             30
Greece                           29
Ecuador                     

In [50]:
"""
Due to class imbalance, transform country into US and non-US.
"""
def transform_country(train,test):
    train.loc[:,'native-country'] = np.where(train.loc[:,'native-country']=='United-States',1,0)
    test.loc[:,'native-country'] = np.where(test.loc[:,'native-country']=='United-States',1,0)
    return train,test

train,test = transform_country(train,test)

In [51]:
# sanity check
train.head()

,age,fnlwgt,education,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income,...,Farming-fishing,Handlers-cleaners,Machine-op-inspct,Other-service,Priv-house-serv,Prof-specialty,Protective-serv,Sales,Tech-support,Transport-moving
0,0.042796,-1.062722,1.127591,1,0,0.146092,-0.218586,-0.077734,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0.880288,-1.007871,1.127591,1,0,-0.147445,-0.218586,-2.331531,1,0,...,0,0,0,0,0,0,0,0,0,0
2,-0.033340,0.244693,-0.657451,1,0,-0.147445,-0.218586,-0.077734,1,0,...,0,1,0,0,0,0,0,0,0,0
3,1.108695,0.425240,-1.549972,0,0,-0.147445,-0.218586,-0.077734,1,0,...,0,1,0,0,0,0,0,0,0,0
4,-0.794697,1.406658,1.127591,0,1,-0.147445,-0.218586,-0.077734,0,0,...,0,0,0,0,0,1,0,0,0,0


In [66]:
# rearrange columns
train_cols = list(train.columns)
train_cols.remove('income')
train_cols.append('income')
train = train[train_cols]
test = test[train_cols]

In [68]:
# finished 1st preprocessing
train.to_csv('../Data/adult_train_preprocessed.csv',index=None)
test.to_csv('../Data/adult_test_preprocessed.csv',index=None)